# Movie Recommender System

A project to predict the rating of unrated movies from a dataset containing large  samples of ratings from users for different movies. The project uses dimensionality reduction to reduce the number of redudant features and understand an underlying pattern.

I've used Spectral Value Decomposition to reduce the high dimensional matrix into its components and Low Rank Approximation to extract important features of 20. 

1. Load the MovieLens data and Convert the data into a matrix (for now, let it be dense)
2. (Optional: if the data is too big to handle, choose a subset of the ratings)
3. Leave out some non zero entries as a test set
4. Perform normalization 
5. Perform SVD
6. Compute the low rank ratings matrix according to the basic latent factor model
7. Test performance against the test data that is left out

## Importing Libraries

In [5]:
import pandas as pd
import numpy as np
import copy
from scipy.linalg import svd

## Loading the dataset


In [2]:
########################################################################################


#Reading File
ratings_list = [i.strip().split(",") for i in open('ml-latest-small/ratings.csv', 'r').readlines()]

#Coverting to DataFrame
ratings_df = pd.DataFrame(ratings_list[1:], columns = ['UserID', 'MovieID', 'Rating', 'Timestamp'], dtype = int)
ratings_df = ratings_df.apply(pd.to_numeric)


#Pivoting to a more easy to use DataFrame
R = ratings_df.pivot(index = 'UserID', columns ='MovieID', values = 'Rating').fillna(0)
print("Shape of R : ", R.shape)
display(R.head(5))
########################################################################################

Shape of R :  (610, 9724)


MovieID,1,2,3,4,5,6,7,8,9,10,...,193565,193567,193571,193573,193579,193581,193583,193585,193587,193609
UserID,,,,,,,,,,,,,,,,,,,,,
1,4.0,0.0,4.0,0.0,0.0,4.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
5,4.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


## Train-Test Split

Randomly choosing about 2000 non-zero entries in R (2000 known ratings given by users to certain items) and create the test matrix B. Define A to be the rest of the non-zero entries. Essentially, A + B should be same as R. After the next few lines of code, you should have two matrices A and B. 

Note: A and B should have no common non-zero entry. The task here is to predict the zero (unknown) entries in A and test them against the available non-zero (known) entries in B.

In [3]:
########################################################################################
def get_AB():
    
    #Retrieving 2000 random Non Zero Entries from Original Non Pivotted DataFrame
    ratings_df_test = ratings_df.sample(n=2000,random_state=42)
    pre_B = ratings_df_test.pivot(index = 'UserID', columns ='MovieID', values = 'Rating')

    #Obtaining B Matrix
    B = (pre_B + R)/2
    print(B.shape)
    
    display(B.head(5))
    
    #Retrieving rest of entries from Original Non Pivotted DataFrame
    temp = ratings_df + ratings_df_test
    ratings_df_train = ratings_df.loc[temp.sum(axis=1)==0,:] 
    Pre_A = ratings_df_train.pivot(index = 'UserID', columns ='MovieID', values = 'Rating')
    
    #Obtaining A Matrix
    A = (Pre_A + R)/2
    
    print(A.shape)
    
    display(A.head(5))
    
    return A,B,ratings_df_test

A, B, ratings_df_test = get_AB()
########################################################################################

(610, 9724)


MovieID,1,2,3,4,5,6,7,8,9,10,...,193565,193567,193571,193573,193579,193581,193583,193585,193587,193609
UserID,,,,,,,,,,,,,,,,,,,,,
1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


(610, 9724)


MovieID,1,2,3,4,5,6,7,8,9,10,...,193565,193567,193571,193573,193579,193581,193583,193585,193587,193609
UserID,,,,,,,,,,,,,,,,,,,,,
1,4.0,NaN,4.0,NaN,NaN,4.0,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5,4.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


## Normalizing and centering

Please note that in A, B or R, the zero entries do not mean the corresponding ratings are zero. They are just unknown. Hence, you need to impute the missing (zero now) entries of A by the average of the known entries (row or column depending on how you have loaded your matrix) for each item. Call this matrix A1.

Then, subtract the average rating given by each user from the corresponding row (or column). Call this matrix A2.

In [4]:
########################################################################################
def Norm_center(A):
    #Replacing NaN values with column averages
    temp0 = A.apply(lambda x: x.fillna(x.mean()),axis=1)
    
    #Replacing NaN values with row averages
    temp1 = temp0.apply(lambda x: x.fillna(x.mean()),axis=0)
    
    #Centering data around row average
    temp2 = temp1.apply(lambda x: x - x.mean(),axis=0)
    
    return(temp1, temp2)

A1, A2 = Norm_center(A)
########################################################################################

## Computing SVD 

Compute SVD of the matrix A2.

In [6]:
########################################################################################
U, s, VT = svd(A2)
########################################################################################

## Computing the low rank approximation 

Compute the low rank approximation of A2. The choice of k is up to you, but we would encourage you to try out a few different k and see which works better. Mention your experience in the readme report.

In [7]:
########################################################################################
def choose_k_rank(U, s, VT, k):
    s_k = copy.deepcopy(s[:k])
    zero_pad = np.zeros((len(s_k),len(VT)-len(s_k)))
    s_k_final = np.concatenate((np.diag(s_k), (zero_pad)), axis = 1)
    
    U_k = copy.deepcopy(U[:,:k])
    return s_k_final, U_k

def low_rank(k):
    s_k, U_k = choose_k_rank(U, s, VT, k)
    A_k = np.matmul((np.matmul(U_k, s_k)),VT)
    return A_k


#Choosing k = 20
Ak = low_rank(20)

Ak = pd.DataFrame(Ak)

display("A2 shape : ", A2.shape)
display("Ak shape : ", Ak.shape)
########################################################################################

'A2 shape : '

(610, 9724)

'Ak shape : '

(610, 9724)

## Testing the performance

Compute the root-mean-squared-error (RMSE) against the known ratings to test how the method performed, using A1 (which predicts ratings just by average known ratings) and Ak against B.

The RMSE is the square root of the average squared errors between the known ratings and the predicted ratings. For our set up, consider only the pairs of (user,item) pairs (i,j) for which B[i,j] > 0. Then compute the square root of the average squared difference between A2 and B (also Ak and B) only for this set of pairs (i,j). 

$$RMSE(A2, B) = \sqrt{\frac{\sum_{i,j: B_{ij} > 0}(A2_{ij} - B_{ij})^2}{\sum_{i,j: B_{ij} > 0} 1}}.$$

$$RMSE(Ak, B) = \sqrt{\frac{\sum_{i,j: B_{ij} > 0}(Ak_{ij} - B_{ij})^2}{\sum_{i,j: B_{ij} > 0} 1}}.$$

In [8]:
########################################################################################
RMSE_A2 = ((np.sqrt(((A2-B)**2).sum().sum()))/(np.sqrt(ratings_df_test.shape[0])))
RMSE_Ak = ((np.sqrt(((Ak-B)**2).sum().sum()))/(np.sqrt(ratings_df_test.shape[0])))
print("RMSE with A2 is %f." %RMSE_A2)
print("RMSE with Ak is %f." %RMSE_Ak)


rmse = []
for k in range(20, 400, 50):
    A_k = pd.DataFrame(low_rank(k))
    ak_rmse = ((np.sqrt(((A_k-B)**2).sum().sum()))/(np.sqrt(ratings_df_test.shape[0])))
    print("For %i RMSE = " %k, ak_rmse)
########################################################################################

RMSE with A2 is 3.804001.
RMSE with Ak is 3.205131.
For 20 RMSE =  3.205131087555014
For 70 RMSE =  3.2073046938129335
For 120 RMSE =  3.2079542907357133
For 170 RMSE =  3.208030116727163
For 220 RMSE =  3.208575842617654
For 270 RMSE =  3.208388812876396
For 320 RMSE =  3.2090861823580177
For 370 RMSE =  3.2093424878875303
